In [1]:
import underworld as uw
import math
from underworld import function as fn
import glucifer.pylab as plt

#linearMesh = uw.mesh.FeMesh_Cartesian(('Q2','dPc1'), (32,32), (0.,0.), (1.,1.))
linearMesh = uw.mesh.FeMesh_Cartesian('Q2Q1', (32,32), (0.,0.), (1.,1.))
constantMesh = linearMesh.subMesh

velocityField = uw.fevariable.FeVariable(linearMesh,2)
velocityField.data[:] = (0.,0.)

pressureField = uw.fevariable.FeVariable(constantMesh,1)
pressureField.data[:] = 0.

# freeslip
IWalls = linearMesh.specialSets["MinI_VertexSet"] + linearMesh.specialSets["MaxI_VertexSet"]
JWalls = linearMesh.specialSets["MinJ_VertexSet"] + linearMesh.specialSets["MaxJ_VertexSet"]
freeslip = uw.conditions.DirichletCondition(velocityField, (IWalls, JWalls))


solA = fn.analytic.SolCx()
# solA = fn.analytic.SolKx()
#solA = fn.analytic.SolKz()

AttributeError: 'module' object has no attribute 'False_swigconstant'

In [2]:
stokesSystem = uw.systems.Stokes(velocityField,pressureField,solA.viscosityFn,solA.bodyForceFn,conditions=[freeslip,], rtolerance=1.e-5)


In [3]:
# can optionally set penalty this way
solver=uw.systems.Solver(stokesSystem, penalty=1.0)

In [4]:

solver.options.A11.ksp_rtol=1e-4
solver.options.scr.ksp_rtol=1e-3

solver.options.main.Q22_pc_type='uwscale'

In [5]:
solver.options.mg.active=False
solver.options.A11.set_direct()
solver.options.A11.list()

In [6]:
from libUnderworld import petsc
petsc.OptionsPrint()

In [7]:
solver.solve()

In [8]:
# can set penalty dynamically this way
solver.options.main.penalty=0.5
solver.solve()

In [9]:
solver.options.mg_accel.list()

In [10]:
solver.options.main.list()
solver.options.mg_accel.mg_accelerating_smoothing=True

In [11]:
# don't have to specify penalty again. Will use currently set value.
solver.solve()

In [13]:
solver.options.mg_accel.mg_smoothing_adjust_on_convergence_rate=True

In [14]:
solver.solve()

In [15]:
from libUnderworld import petsc

In [16]:
#petsc.OptionsClear()

In [17]:
petsc.OptionsPrint()

In [18]:
solver.options.main.Q22_pc_type='uw'
solver.solve(penalty=0)

In [19]:
solver.options.main.change_backsolve=True
solver.options.backsolveA11.ksp_view=None

In [20]:
solver.solve()

In [21]:
solver.options.backsolveA11.list()

In [22]:
solver.options.backsolveA11.help()
